# Transform GIS coordinates Tester


In [2]:
'''
    WARNING CONTROL to display or ignore all warnings
'''
import warnings; warnings.simplefilter('ignore')     #switch betweeb 'default' and 'ignore'
import traceback

''' Set debug flag to view extended error messages; else set it to False to turn off debugging mode '''
debug = True


## Instantiate Classes

In [17]:
import os
import sys

proj_dir = os.path.abspath(os.pardir)
sys.path.insert(1,proj_dir.split('rezaware/')[0])
from rezaware.modules.etl.transform import gisCoordinates as gis
from rezaware.modules.lib.spark import execSession as spark

''' restart initiate classes '''
if debug:
    import importlib
#     db = importlib.reload(db)
    gis=importlib.reload(gis)

__desc__ = "read and write files from and to postgresql database"
clsSpark = spark.Spawn(
    desc=__desc__,
)
if clsSpark.session:
    clsSpark._session.stop
clsGIS = gis.dataWorkLoads(desc=__desc__)
print("\n%s class initialization and load complete!" % __desc__)

All functional GISCOORDINATES-libraries in TRANSFORM-package of ETL-module imported successfully!
execSession Class initialization complete
execSession Class initialization complete
gisCoordinates Class initialization complete

read and write files from and to postgresql database class initialization and load complete!


## Transform Decimal to Degree coordinates

In [18]:
from pyspark.sql import functions as F

# _from_to_coord = ('decimal','degrees')
# _col_lst = ['GEO LONGITUDE','GEO LATITUDE']
_coord_cols_dict = {
    "LATITUDE" : {
        "COLATTR" : 'GEO LATITUDE',
        "COMPATTR": 'NORTH/SOUTH',
    },
    "LONGITUDE" : {
        "COLATTR" : 'GEO LONGITUDE',
        "COMPATTR": 'EAST/WEST',
    },
}
kwargs = {
    "DEGLONATTR" : 'deg_lon',
    "DEGLATATTR" : 'deg_lat',
}

_from_to_coord = ('decimal','degrees')
_convert_data_sdf = clsGIS.convert_coordinates(
    data = grps_df, # non empty dataframe with coordinates columns
    from_to=_from_to_coord,  # ordered pair defines convert from to method
    coord_cols=_coord_cols_dict,  # x, y, z, column names, in that order
    **kwargs,  # additional behaviour changing key value pairs
)

_convert_data_sdf.select(F.col('COMPANY 1'), F.col('CITY'),
                         F.col('GEO LONGITUDE'), F.col('GEO LATITUDE'),
                         F.col('deg_lon'),F.col('deg_lat')).show(n=2, truncate=False)

kwargs = {
    "GEOMATTR" : 'geometry',
}
_from_to_coord = ('decimal','geometry')
_convert_data_sdf = clsGIS.convert_coordinates(
    data = grps_df, # non empty dataframe with coordinates columns
    from_to=_from_to_coord,  # ordered pair defines convert from to method
    coord_cols=_coord_cols_dict,  # x, y, z, column names, in that order
    **kwargs,  # additional behaviour changing key value pairs
)
_convert_data_sdf.select(F.col('COMPANY 1'), F.col('CITY'),
                         F.col('GEO LONGITUDE'), F.col('GEO LATITUDE'),
                         F.col('geometry'),).show(n=2, truncate=False)

+---------+------+------------------+------------------+-------------+-------------+
|COMPANY 1|CITY  |GEO LONGITUDE     |GEO LATITUDE      |deg_lon      |deg_lat      |
+---------+------+------------------+------------------+-------------+-------------+
|Hotel    |Luanda|13.234682411130052|-8.816315687788876|13º14'4.86"E |8º48'58.74"S |
|Hostel   |Giza  |31.214035749435425|29.89082264132472 |31º12'50.53"E|29º53'26.96"N|
+---------+------+------------------+------------------+-------------+-------------+
only showing top 2 rows

+---------+------+------------------+------------------+---------------------------------------------+
|COMPANY 1|CITY  |GEO LONGITUDE     |GEO LATITUDE      |geometry                                     |
+---------+------+------------------+------------------+---------------------------------------------+
|Hotel    |Luanda|13.234682411130052|-8.816315687788876|POINT (13.234682411130052 -8.816315687788876)|
|Hostel   |Giza  |31.214035749435425|29.8908226413247

## Read GIS dataset

In [4]:
import pandas as pd
fpath = os.path.join(proj_dir.split('rezaware/')[0],"mining/data/property/hotel_groups.csv")

grps_df = pd.read_csv(fpath, header=1)
grps_df

/home/nuwan/workspace/HERO/mining/data/property/hotel_groups.csv


,ID,SEGMENTATION,COMPANY 1,COMPANY 2,COMPANY 3,STREET,BUILDING,STREET2,P.O. BOX,POSTAL CODE,...,FIRST NAME,FAMILY NAME,FUNCTION,PERSONAL TELEPHONE,PERSONAL EMAIL ADDRESS,GEO LATITUDE,NORTH/SOUTH,GEO LONGITUDE,EAST/WEST,GEO ACCURACY
0,"5,272,478",H,Hotel,Skyna Luanda,NaN,Rua de Portugal,29.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,-8.816316,S,13.234682,E,BUILDING
1,"8,835,928",HO,Hostel,Sakkara Inn,NaN,Sakkara Road,20.0,NaN,NaN,NaN,...,NaN,NaN,NaN,+20-10-04906656,NaN,29.890823,N,31.214036,E,BUILDING
2,"8,011,712",R,Resort,Sea Gull Hurghada,Owner & Operator: El Noras for Tourism Investment,Sheraton Road,NaN,Sakkala,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,27.216327,N,33.839992,E,BUILDING
3,"8,004,273",R,Resort,Grand Oasis,NaN,Sharks Bay,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,27.957088,N,34.390238,E,BUILDING
4,"8,575,298",H,Hotel,Tranquility Bay,NaN,Bolands,NaN,NaN,JH94,NaN,...,NaN,NaN,NaN,NaN,NaN,17.066816,N,-61.887601,W,BUILDING
5,"3,008,365",HR,Beach Hotel & Spa,Accra,NaN,Rockley Beach,NaN,NaN,73W,BB15139,...,Jon,Martineau,GM,NaN,NaN,13.074867,N,-59.589721,W,BUILDING
6,"8,008,485",R,Beach Resort,Sunshine Suites,NaN,Esterley Tibbetts Highway,1465.0,Seven Mile Beach,30095 SMB,KY1-1201,...,NaN,NaN,NaN,NaN,NaN,19.338550,N,-81.378420,W,BUILDING
7,"8,054,465",R,Resort,East Winds Inn,NaN,La Brelotte Bay,NaN,NaN,1477,NaN,...,NaN,NaN,NaN,NaN,NaN,14.053917,N,-60.972225,W,BUILDING
8,"8,080,780",R,Resort,The Buccaneer,NaN,Estate Shoys,5007.0,NaN,25200,00820-4536,...,NaN,NaN,NaN,NaN,NaN,17.754312,N,-64.681291,W,BUILDING
9,"3,008,436",HR,Hotel,Posada Viena,NaN,Marsella,28.0,NaN,NaN,06600,...,Jose Antonio Pontón,Posada,CEO,NaN,NaN,19.426086,N,-99.157766,W,BUILDING
